In [ ]:
# Balazs Balogh - 2019-03-01

import pandas as pd
import sqlite3
import sqlalchemy

stops_orig = pd.read_csv('https://raw.githubusercontent.com/DatasRev/workshop-prep/master/08_Python_and_SQL/stops.txt')
stops_orig.head()

Miután betöltöttük a szükséges könyvtárakat, és a sample filet, megvizsgáljuk, hogy a nekünk kellő mezők töltve vannak-e. Az első négy mezőben nincs NaN érték, így nem szükséges adatot tisztítani.

In [ ]:
stops_orig.info()

Az első négy oszlopot külön változóba mentjük.

In [ ]:
stops = stops_orig[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stops.head()

Hogy jól be tudjuk lőni, hogy mekkora mezőméretek kellenek, megnézzük a stop_name leghosszabb sorát.
Az SQLite épp nem kezeli a VARCHAR mögötti számértéket (pl. VARCHAR(255)), mivel TEXT-ként kezeli, de más adatbázisnál jól jöhet.
Akár egy for loopot is lehet rá írni, hogy mondja meg az összes object típusú oszlopról, hogy melyik a leghosszabb érték.

In [ ]:
stops.stop_name.map(len).max()  #megnézi, hogy melyik a leghosszabb string az oszlopban

Defaultban a notebook könyvtárába rakja az adatbázist is. Egy if-else kell, hogy ha többször futtatjuk, akkor továbbmehessen a notebook futtatás. A notebook ezen része példa arra, hogy hogy insertálunk adatbázisba adatot. 

In [ ]:
db = sqlite3.connect('bkk_gtfs.db')
cursor = db.cursor()

# ez az elso lepes, elkeszitjuk a tablat, ha mar van, akkor tovabblepunk.
if cursor.execute('select count() from bkk_stops') == 0:
    
    cursor.execute(
    '''
    CREATE TABLE bkk_stops(
           stop_id TEXT PRIMARY KEY,
           stop_name TEXT,
           stop_lat REAL,
           stop_lon REAL)
    '''
    )

    db.commit()

    # aztan beinsertalunk, .values.tolist()-el kell listbe rakni a dataframe-et.
    cursor.executemany('''INSERT INTO bkk_stops(stop_id, stop_name, stop_lat, stop_lon) VALUES(?,?,?,?)''',
                      stops.values.tolist())
    db.commit()
    
else:
    print("Table is already present.")

Teszteljük egy lekérdezéssel, a fetchall() parancs adja vissza az összes sort, de kapott egy tizes limitet.

In [ ]:
cursor.execute('SELECT * FROM bkk_stops LIMIT 10').fetchall()

Beolvassuk egy DataFrame-be a teljes bkk_stops táblát. A db változót kell odaadni neki, abban van a kapcsolat az SQLite adatbázissal.

In [ ]:
bkk_stops = pd.read_sql_query('SELECT * FROM bkk_stops', db)

bkk_stops.info()

In [ ]:
bkk_stops.head()

Listába kell rakni a dataframe-et, hogy olvasni tudja a folium

In [ ]:
stops_latlon = bkk_stops[['stop_lat', 'stop_lon']].values.tolist()
stops_latlon[:10]

Folium importálása, és egy üres map létrehozása egy budapesti koordinátával a középpontban. A popup attribútuma a Markernek teszi rá a megálló nevét a jelzésre.
Ezután jön az 5000 megálló kirajzolása, ami a méreténél fogva igényli a MarkerCluster-t, hogy összesítsen megállókat.
Opcionálisan html-be mentjük, majd ezt hívjuk meg.
A térképet is az anaconda mappájába fogja rakni.

In [ ]:
import folium
from folium.plugins import MarkerCluster

folium_map = folium.Map(location=[47.500368, 19.103406],
                        zoom_start=13,
                        tiles="cartodbpositron")

In [ ]:
folium_map = folium.Map(location=[47.500368, 19.103406],
                        zoom_start=13,
                        tiles="cartodbpositron")

marker_cluster = MarkerCluster().add_to(folium_map)

for point in range(0, len(stops_latlon)):
    folium.Marker(stops_latlon[point], popup=stops['stop_name'][point]).add_to(marker_cluster)

folium_map.save("base_map.html")

In [ ]:
from IPython.display import IFrame
IFrame("base_map.html", width=700, height=450)